<center>
  <h1>
    JUEGO DE DOMINÓ USANDO MONTE CARLO TREE SEARCH COMO INTELIGENCIA DE MÁQUINA
  </h1>
</center>

<p>
  <h4>INTEGRANTES DEL PROYECTO</h4>
  <ul>
    <li>
      Diana Alejandra Herrera Blanco 
    </li>
    <li>
      Jhoan Manuel Díaz Higuera
    </li>
    <li>
      Karen Daniela Rodríguez Martínez
    </li>
    <li>
      Víctor Alfonso Mantilla
    </li>
  </ul>
</p>


In [2]:
#Necesario para usar anytree
pip install anytree

     |████████████████████████████████| 51kB 8.0MB/s 


<center>
  <h2>
    1. Introducción
  <h2>
</center>

<p >
A través del avance tecnológico, se ha tratado de simular la inteligencia humana en los computadores. La inteligencia artificial se ha transformado en el boom de la computación, debido a que de esta manera se ha mejorado la forma en que “las máquinas aprendan de la experiencia, se adapten a nuevas entradas y realicen tareas similares a las de los humanos" (Gartner, 2019).
  De aqui podemos entender el porqué se ha vuelto tendencia el tema de la máquina vs el humano.
  Este proyecto está enfocado a la simulación del juego **DOMINÓ** en el cual, se enfrenta la máquina vs una persona. Se va a tratar de simular una cantidad de juegos, a través movimientos aleatorios del humano y la máquina tratará de usar MCTS para realizar su mejor movimiento.
</p>

<center>
  <h2>
    2. Motivación
  <h2>
</center>
<p>
Nuestras principales motivaciones para realizar este proyecto son: 
El aumento en el uso de Monte Carlo Tree Search en los juegos de mesa para la toma de decisiones de la maquina, esto se debe a la gran popularidad que adquirió gracias a la victoria de Alpha Go, una inteligencia artificial, sobre el campeón del juego de mesa GO.
La nula implementación del Monte Carlo Tree Search para la simulación del juego de mesa domino.
</p>

<center>
  <h2>
    3. Dominó
  </h2>
</center>
<p>
  El juego de dominó fue inventado en China en el siglo XII, y hasta el momento hay muchas variaciones del mismo. La que más se conoce en Latinoamérica y Colombia es la variación de 28 fichas, donde la ficha más grande es doble seis, y la mínima es doble cero.
</p>

<p>
Para este trabajo se implementa una versión sencilla del juego, con tan solo dos jugadores (humano y computador) cuyas reglas son:
  <ul>
    <li> Cada jugador inicia con siete fichas al azar.
    <li> Inicia colocando una ficha sobre la "mesa" el jugador humano. Siempre inicia el humano y puede colocar cualquier ficha.
    <li> En los siguientes turnos, cada jugador debe poner una ficha que coincida con alguno de los dos extremos de las fichas en la mesa.
    <li> Si un jugador no tiene una posible ficha para poner, debe tomar una ficha de la pila (las fichas no repartidas al inicio), si esta ficha tomada corresponde a algún extremo, la debe poner, si no coincide, se la queda.
    <li> Si la pila de fichas está vacía o la ficha que tomó de esta no es válida para jugar en el turno actual, el jugador debe pasar.
    <li> Gana el juego el jugador que primero se quede sin fichas.
    <li> Si ambos jugadores pasan en un turno y la pila ya está vacía, termina el juego y gana el jugador cuya suma de los puntos de sus fichas sea menor, o empatan si las sumas son iguales.
  </ul>
</p>

<p>
  A continuación se muestra el algoritmo que permite implementar la versión de dominó planteada. Ejecutando las celdas siguientes se puede usar esta implementación para jugar. La implementación usa la consola como interfaz de interacción.
</p>

In [0]:
import numpy as np
from copy import deepcopy as dc
import sys
from datetime import datetime as dt
from anytree import Node, RenderTree, AsciiStyle, LevelOrderIter

In [0]:
#domino game simulation functions
def add_tile(tile,U,side=None):
  if len(U)==0:
    U.append(tile)
  elif side=="left":
    lv=get_left_value(tile)
    rv=get_right_value(tile)
    ti=get_tile_index(tile)
    for i in range(len(U)):
      cln=get_left_neighbour(U[i])
      cti=get_tile_index(U[i])
      if cln==0:
        clv=get_left_value(U[i])
        if clv==lv:
          U[i]=set_left_neighbour(U[i],ti)
          tile=set_right_neighbour(tile,cti)
          tile=turn_tile(tile)
          U.append(tile)
          break
        if clv==rv:
          U[i]=set_left_neighbour(U[i],ti)
          tile=set_right_neighbour(tile,cti)
          U.append(tile)
          break
  elif side=="right":
    lv=get_left_value(tile)
    rv=get_right_value(tile)
    ti=get_tile_index(tile)
    for i in range(len(U)):
      crn=get_right_neighbour(U[i])
      cti=get_tile_index(U[i])
      if crn==0:
        crv=get_right_value(U[i])
        if crv==rv:
          U[i]=set_right_neighbour(U[i],ti)
          tile=set_left_neighbour(tile,cti)
          tile=turn_tile(tile)
          U.append(tile)
          break
        if crv==lv:
          U[i]=set_right_neighbour(U[i],ti)
          tile=set_left_neighbour(tile,cti)
          U.append(tile)
          break
  else:
    prev=len(U)
    lv=get_left_value(tile)
    rv=get_right_value(tile)
    ti=get_tile_index(tile)
    for i in range(len(U)):
      cln=get_left_neighbour(U[i])
      crn=get_right_neighbour(U[i])
      cti=get_tile_index(U[i])
      if cln==0:
        clv=get_left_value(U[i])
        if clv==lv:
          U[i]=set_left_neighbour(U[i],ti)
          tile=set_right_neighbour(tile,cti)
          tile=turn_tile(tile)
          U.append(tile)
          break
        elif clv==rv:
          U[i]=set_left_neighbour(U[i],ti)
          tile=set_right_neighbour(tile,cti)
          U.append(tile)
          break
      if crn==0:
        crv=get_right_value(U[i])
        if crv==rv:
          U[i]=set_right_neighbour(U[i],ti)
          tile=set_left_neighbour(tile,cti)
          tile=turn_tile(tile)
          U.append(tile)
          break
        if crv==lv:
          U[i]=set_right_neighbour(U[i],ti)
          tile=set_left_neighbour(tile,cti)
          U.append(tile)
          break
    if(prev==len(U)):
      print("Error in add_tiles!")
      print(U)
  return U

def all_tiles():#Tile structure: index,turned (0:not turned),left neighbour,right neighbour,value1,value2
  array=[]
  for i in range(28):
    if i<7:
      tile=[i+1,0,0,6,i]
    elif i<13:
      tile=[i+1,0,0,5,i%7]
    elif i<18:
      tile=[i+1,0,0,4,i%13]
    elif i<22:
      tile=[i+1,0,0,3,i%18]
    elif i<25:
      tile=[i+1,0,0,2,i%22]
    elif i<27:
      tile=[i+1,0,0,1,i%25]
    else:
      tile=[i+1,0,0,0,0]
    array.append(tile)
  return array

def ask_for_move(limit):  #elige la ficha para jugar
  move=0
  while move==0:
    print("Please select a tile to play:")
    cand=int(input())
    if cand>0 and cand<=limit:
      move=cand
    else:
      print("Invalid tile.")
  print("")
  return move-1

def ask_for_side(extremes):
  side=-1
  while side<0:
    print("Please choose a side to play:")
    print("Left extreme ("+str(extremes[0])+") write 1.")
    print("Right extreme ("+str(extremes[1])+") write 2.")
    cand=int(input())
    if cand==1 or cand==2:
      side=cand
    else:
      print("Invalid side.")
  print("")
  return side-1

def put_tile(tile,U,extremes,strat="random"): #añade la ficha del jugador en un extremo del juego
  left_value=get_left_value(tile)
  right_value=get_right_value(tile)
  if (left_value==extremes[0] and right_value==extremes[1]) or (left_value==extremes[1] and right_value==extremes[0]):
    if strat=="random":
        side=np.random.randint(2)
    elif strat=="human":
        side=ask_for_side(extremes)
    if side==0:
        U=add_tile(tile,U,"left")
    elif side==1:
        U=add_tile(tile,U,"right")
  else:
    U=add_tile(tile,U)
  return U

def extreme_tiles_numbers(U):
  extremes=[-1,-1]
  for tile in U:
    if get_left_neighbour(tile)==0:
      extremes[0]=get_left_value(tile)
    if get_right_neighbour(tile)==0:
      extremes[1]=get_right_value(tile)
  if extremes[0]>=0 and extremes[1]>=0:
    return extremes
  else:
    print("Error in extremes!")
    print(U)

def get_left_neighbour(tile):
  return tile[1]

def get_left_value(tile):
  return tile[3]

def get_right_neighbour(tile):
  return tile[2]

def get_right_value(tile):
  return tile[4]

def get_tile_index(tile):
  return tile[0]

def get_tiles(tiles,amount):
  mix_tiles(tiles)
  l=len(tiles)
  if l>=amount:
    extracted=[]
    for i in range(amount):
      tile=tiles[np.random.randint(l-i)]
      tiles.remove(tile)
      extracted.append(tile)
    return extracted,tiles

def graph_current_game(U):
  copy=U.copy()
  if len(copy)==1:
    text="|"+str(get_left_value(copy[0]))+":"+str(get_right_value(copy[0]))+"|"
  elif len(U)>1:
    left_extreme=0
    arranged_tiles=[]
    text=""
    while len(copy)>0:
      for tile in copy:
        cln=get_left_neighbour(tile)
        if cln==left_extreme:
          arranged_tiles.append(tile)
          copy.remove(tile)
          left_extreme=get_tile_index(tile)
          break
    for tile in arranged_tiles:
      text+="|"+str(get_left_value(tile))+":"+str(get_right_value(tile))+"|"
  print(text)
  print("")
# se grafican las fichas de cada jugador 
def graph_tiles(P,limit=0):
  text=""
  for i in range(len(P)):
    text+="|"+str(get_left_value(P[i]))+"|"
  text+="\n"
  for i in range(len(P)):
    text+="|"+str(get_right_value(P[i]))+"|"
  text+="\n"
  for i in range(limit):
    text+=" "+str(i+1)+" "
  print(text)
  print("")

def mix_tiles(tiles):
  return np.random.shuffle(tiles)

def playable_tiles(P,extremes): #devuelve las fichas validas del jugador para el juego
  limit=0
  arranged_tiles=[]
  change=True
  while change:
    change=False
    for tile in P:
      left_value=get_left_value(tile)
      right_value=get_right_value(tile)
      if left_value in extremes or right_value in extremes:
        arranged_tiles.append(tile)
        P.remove(tile)
        change=True
        limit+=1
        break
  for tile in P:
    arranged_tiles.append(tile)
  return limit, arranged_tiles

def set_left_neighbour(tile, index):
  tile[1]=index
  return tile

def set_right_neighbour(tile,index):
  tile[2]=index
  return tile

def start_game():
  S=all_tiles()
  P,S=get_tiles(S,7)
  C,S=get_tiles(S,7)
  return P,C,S

def sum_tile_values(P):
  summ=0
  for tile in P:
    summ+=get_left_value(tile)+get_right_value(tile)
  return summ

def turn_tile(tile):
  aux=tile[3]
  tile[3]=tile[4]
  tile[4]=aux
  return tile

def get_UPS(U,P,S):
    UPS=[]
    extremes=extreme_tiles_numbers(U)
    limit,P=playable_tiles(P,extremes)
    if limit==0:
        if len(S)>0:
            for s in S:
                NU=dc(U)
                NP=dc(P)
                NS=dc(S)
                NS.remove(s)
                NP.append(s)
                UPS.append([NU,NP,NS])
            return True,UPS
        else:
            UPS=None
    else:
        for l in range(limit):
            NU=dc(U)
            NP=dc(P)
            NS=dc(S)
            NU=put_tile(NP[l],NU,extremes)
            NP.remove(NP[l])
            UPS.append([NU,NP,NS])
        return False,UPS


<center>
  <h2>
    4. Monte Carlo Tree Search
  <h2>
</center>
<p>
Es un método para toma óptima de decisiones en problemas de inteligencia artificial. Combina la generalidad de simulaciones aleatorias con la precisión de una búsqueda en el árbol de posibilidades. 
MCTS tiene cuatro procesos básicos: selección,expansión, simulación y propagación hacia atrás, los cuales se realizan durante cada iteración.

El MCTS que usa la máquina, se basa principalmente en simular todas las posibles jugadas, y ver su impacto, que puede ser tanto positivo como negativo y escoge la que más le convenga para ganar la partida.

El siguiente algoritmo simula una jugada. Para decidir los movimientos, se usa una distribución uniforme en el espacio de jugadas válidas.
</p>


![texto alternativo](https://www.cs.us.es/~fsancho/images/2018-01/mcts.png)

In [0]:
def simulate_game(U=[],P=[],C=[],S=[],T=False,R=True,player_strat="random",pc_strat="random",iterations=1000,c=2,graph=False): 
    #R es el que verifica a quien pertenece el turno (P/C) 
    if len(U)==0 and len(P)==0 and len(C)==0 and len(S)==0:
        U=[]  #fichas que se han jugado (el juego)
        P=[]  #fichas del jugador
        C=[]  #fichas del computador
        S=[]  #fichas del stock
        T=False
        #Se genera el juego, distruyendo las fichas entre PLAYER, COMPUTER Y STOCK
        P,C,S=start_game()
        #Se grafican las fichas de cada jugador y stock
        if graph:
            graph_tiles(S)
            graph_tiles(P)
            graph_tiles(C)
    #se inicializa score=-1 porque la partida no ha iniciado        
    score=-1
    while score==-1:
        if player_strat=="human" and (len(U))>0: #se verifica si existe al menos una jugada
            if R:
                print("Player turn: Current game state")
                print("")
            else:
                print("Computer turn: Thinking...")
                print("")
            #Muestra la jugada actual    
            graph_current_game(U)
            print("")
            print("Computer tiles left:",len(C)) #muestra la cantidad de fichas que tiene la maquina
            print("")
        elif player_strat=="human": #Inicia el juego
            print("======================================")
            print("")
            print("DOMINO GAME")
            print("")
            print("Current Computer Strategy:",pc_strat)
            print("")
            print("======================================")
            print("")
        aux=None  
        if R: #turno del humano
            aux=player_turn(U,P,C,S,strat=player_strat)
            if isinstance(aux,int):
                score=aux
            else:
                U,P,C,S,T=aux
            R=not R
            if graph:
                graph_current_game(U)
                graph_tiles(S)
                graph_tiles(P)
                graph_tiles(C)
        else:
            if player_strat=="human":
                now=dt.now()
            aux=pc_turn(U,P,C,S,T,strat=pc_strat,iterations=iterations,c=c) #jugada de la maquina
            if player_strat=="human":
                print(dt.now()-now)
            if isinstance(aux,int):
                score=aux
            else:
                U,P,C,S=aux
            R=not R
            if graph:
                graph_current_game(U)
                graph_tiles(S)
                graph_tiles(P)
                graph_tiles(C)
    if player_strat=="human" and score==0:
        print("Player wins!")
        print("")
        graph_current_game(U)
    elif player_strat=="human" and score==1:
        print("Computer wins!")
        print()
        graph_current_game(U)
    return score

def random_strategy(U,P,limit,extremes):
    n=np.random.randint(limit)
    U=put_tile(P[n],U,extremes)
    P.remove(P[n])
    return U,P

def max_strategy(U,P,limit,extremes):
    maxx=0
    n=0
    for i in range(len(P[:limit])):
        cand=sum_tile_values([P[i]])
        if cand>maxx:
            maxx=cand
            n=i
    U=put_tile(P[n],U,extremes)
    P.remove(P[n])
    return U,P
# estrategia del humano
def human_strategy(U,P,limit,extremes):
    graph_tiles(P,limit)  #grafica las fichas del jugador
    n=ask_for_move(limit) #selecciona la ficha para jugar
    if len(U)==0: 
        U=[P[n]]  #Agrega la ficha seleccionada a la lista del juego
    else:
        U=put_tile(P[n],U,extremes,strat="human") #añade la ficha del jugador en un extremo del juego
    P.remove(P[n])
    return U,P  #retorna el juego con la ficha agregada y la mano del jugador sin la ficha jugada
    
def MCTS_strategy(U,P,C,S,iterations=1000,c=2,simulations=10000):
    state={"name":"S-0",
           "U":dc(U),
           "P":dc(P),
           "C":dc(C),
           "S":dc(S),
           "T":False,
           "L":False,
           "R":False,
           "n":0,
           "t":0}
    result=MCTS(state,iterations=iterations,c=c)
    return result.name["U"],result.name["P"],result.name["C"],result.name["S"]

def fixed_strategy(U,P,extremes,n):
    U=put_tile(P[n],U,extremes)
    P.remove(P[n])
    return U,P

def player_turn(U,P,C,S,strat="random"):  #se ejecuta el turno del jugador humano
    T=False 
    if len(U)==0: #en caso de que sea la primera jugada
        if strat=="random":
            n=np.random.randint(len(P))
            U.append(P[n])  #añade la ficha al juego
            P.remove(P[n])  #quita la ficha del jugador
            return U,P,C,S,T
        elif strat=="human":
            U,P=human_strategy(U,P,len(P),[])   #se decide la estrategia del humano
            return U,P,C,S,T  
    else:
        extremes=extreme_tiles_numbers(U) #obtiene las fichas de los extremos del juego
        limit,P=playable_tiles(P,extremes)  #obtiene las fichas validas del jugador
        if limit==0:  #evalua si tiene fichas validas para jugar
            if len(S)>0:  #evalua si hay fichas en el stock
                if strat=="human":
                    print("No valid tiles to play. Taking one from stock...")
                    print("")
                tile,S=get_tiles(S,1) #obtiene una ficha del stock
                limit,tile=playable_tiles(tile,extremes)  #verifica que la ficha es valida
                if limit==0:  #evalua si tiene fichas validas
                    if strat=="human":
                        print("Invalid tile taken from stock. Player must pass.")
                        print("")
                    P.append(tile[0])
                    T=True  #pasa el jugador 
                else:
                    if strat=="human":
                        print("Valid tile taken from stock. Playing with it...")
                        print("")
                    U=put_tile(tile[0],U,extremes)  #agrega la ficha al juego
            else:
                T=True
        else: #juega con las fichas validas
            if strat=="random":
                U,P=random_strategy(U,P,limit,extremes)
            elif strat=="human":
                U,P=human_strategy(U,P,limit,extremes)
            elif strat=="max":
                U,P=max_strategy(U,P,limit,extremes)
        if len(P)==0:
            return 0
        else:
            return U,P,C,S,T

def pc_turn(U,P,C,S,T,strat="random",iterations=1000,c=2):
    R=False
    extremes=extreme_tiles_numbers(U)
    limit,C=playable_tiles(C,extremes)
    if limit==0:
        if len(S)>0:
            tile,S=get_tiles(S,1)
            limit,tile=playable_tiles(tile,extremes)
            if limit==0:
                C.append(tile[0])
                R=True
            else:
                U=put_tile(tile[0],U,extremes)
        else:
            R=True
    elif limit==1:
        U=put_tile(C[0],U,extremes)
        C.remove(C[0])
    else:
        if strat=="random":
            U,C=random_strategy(U,C,limit,extremes)
        elif strat=="MCTS":
            U,P,C,S=MCTS_strategy(U,P,C,S,iterations=iterations,c=c)
    if len(C)==0:
        return 1
    elif T and R and len(S)==0: #si player y computer pasaron y no hay fichas en stock
        P_sum=sum_tile_values(P)
        C_sum=sum_tile_values(C)
        if P_sum>C_sum:
            return 1
        else:
            return 0
    else:
        return U,P,C,S

def add_children(parent):
    turn=parent.name["R"]
    name=parent.name["name"]
    first=None
    if turn:
        result=get_UPS(dc(parent.name["U"]),dc(parent.name["P"]),dc(parent.name["S"]))
        if result!=None:
            passed,UPS=result
            index=0
            for ups in UPS:
                data={"name":name+"-"+str(index),
                    "U":ups[0],
                    "P":ups[1],
                    "C":dc(parent.name["C"]),
                    "S":ups[2],
                    "T":passed,
                    "L":False,
                    "R":not turn,
                    "n":0,
                    "t":0}
                if first==None:
                    first=Node(data,parent=parent)
                else:
                    Node(data,parent=parent)
                index+=1
        else:
            parent.name["L"]=True #el padre es una hoja
    else:
        result=get_UPS(dc(parent.name["U"]),dc(parent.name["C"]),dc(parent.name["S"]))
        if result!=None:
            passed,UCS=result
            index=0
            for ucs in UCS:
                data={"name":name+"-"+str(index),
                      "U":ucs[0],
                      "P":dc(parent.name["P"]),
                      "C":ucs[1],
                      "S":ucs[2],
                      "T":False,
                      "L":False,
                      "R":not turn,
                      "n":0,
                      "t":0}
                if first==None:
                    first=Node(data,parent=parent)
                else:
                    Node(data,parent=parent)
                index+=1
        else:
            parent.name["L"]=True
    
def backpropagate(state,value):
  state.name["n"]=state.name["n"]+1
  state.name["t"]=state.name["t"]+value
  while not state.is_root:
    state=state.parent
    state.name["n"]=state.name["n"]+1
    state.name["t"]=state.name["t"]+value

def calculate_ucb1(state,p,c=2):
  n=state.name["n"]
  t=state.name["t"]
  if n==0:
    return sys.maxsize
  else:
    return (t/n)+c*np.sqrt(np.log(p)/n)

def rollout(state,simulations=10000):
    return simulate_game(U=dc(state.name["U"]),
                        P=dc(state.name["P"]),
                        C=dc(state.name["C"]),
                        S=dc(state.name["S"]),
                        T=dc(state.name["R"]),
                        R=dc(state.name["T"]))
  
def get_best_child(parent,end_nodes=True,c=2):
  p=parent.name["n"]  
  highest=0
  best_child=None
  for child in LevelOrderIter(parent,maxlevel=2):
    candidate=0
    if child!=parent and (not end_nodes or (end_nodes and not child.name["L"])):
        candidate=calculate_ucb1(child,p,c=c)
    if candidate>highest:
      highest=candidate
      best_child=child
  if best_child==None:
      parent.name["L"]=True
  return best_child

def MCTS(state,iterations=1000,c=2,simulations=10000):
  root = Node(state)
  iters=0
  while iters<iterations and root.name["L"]==False:
    current=root  #el nodo actual es el nodo raiz
    iters+=1
    while not current.is_leaf:  #si no es una hoja
      current=get_best_child(current) #selecciona con UCB el mejor hijo
      if current==None:
        break
    if current!=None: #si es una hoja
      if current.name["n"]==0:
        value=rollout(current,simulations)  #hace una sumulacion de las jugadas posibles y obtiene un score
        backpropagate(current,value)  #asigna el valor de n y t a sus padres
      else:
        current=add_children(current) #si n!=0 agrega hijos
        if current!=None:
          value=rollout(current,simulations)
          backpropagate(current,value)
  #for pre, _, node in RenderTree(root,maxlevel=2):
    #print("%s%s" % (pre, node.name["name"]+": "+str(node.name["n"])+","+str(node.name["t"])))
  return get_best_child(root,end_nodes=False)

In [0]:
score=[simulate_game(pc_strat="MCTS",iterations=10000,c=9) for _ in range(10)]

In [9]:
print(score)
print(np.mean(score))

[1, 1, 1, 0, 1, 0, 1, 1, 0, 1]
0.7


<center>
  <h2>
    5. Resultados
  <h2>
</center>
<p>
 Realizando experimentaciones nuevamente, esta vez cambiando la forma de jugar del computador, de una distribución uniforme a monte carlo tree search, obtenemos que el computador gana el 65.67% de las veces.

Esto es una mejora aproximada del 20% con respecto al método de la distribución uniforme, y se realizan 264 muestras. Estas muestras están muy limitadas con respecto al   poder de cómputo disponible, ya que cada una de estas simulaciones podía llevar incluso hasta una hora y media de ejecución.

Calculamos el error relativo entre el score obtenido y el score esperado, esto nos da un error del 5.75%
</p>

<center>
  <h2>
    6. Conclusiones
  <h2>
</center>
<p>
 Nuestras principales conclusiones son que el método de MCTS es un método efectivo a la hora de ser usado como inteligencia de maquina para simular el juego de dominó, también, concluimos que el número de iteraciones puede variarse para aumentar o disminuir la dificultad en un juego simulado de dominó, además que, la efectividad del método está intrínsicamente relacionada con los recursos computacionales que se tengan a la mano.
</p>

<center>
  <h2>
    7. Trabajo futuro
  <h2>
</center>
<p>
Se espera encontrar una mejora de determinar el valor del parámetro c, para esto se espera, conseguir recursos computacionales mejores y más potentes, para así, disminuir el tiempo de las experimentaciones. Por último, se desea implementar una aplicación que sea totalmente iterativa con el usuario y que utilice nuestro método propuesto. **GRACIAS**.
</p>